# Homework

In [16]:
#importing libraries
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [2]:
# reading the january dataset.
mar_df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")
mar_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00


In [3]:
# nums of rows and columns
mar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3403766 entries, 0 to 3403765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [4]:
# computing the duraion variable
mar_df["duration"] = mar_df.tpep_dropoff_datetime - mar_df.tpep_pickup_datetime

 # convert duration column to mins
mar_df.duration = mar_df["duration"].apply(lambda td: td.total_seconds()/ 60)

In [5]:
# estiamting the standard deviation of duraion variable
mar_df.describe().iloc[:, -1]

count    3.403766e+06
mean     1.688182e+01
std      4.379458e+01
min     -4.762500e+02
25%      7.400000e+00
50%      1.215000e+01
75%      1.965000e+01
max      5.637717e+03
Name: duration, dtype: float64

* The standard deviation for duration is 4.38

In [6]:
# fraction of recorda left dropping outliers
((mar_df.duration >= 1) & (mar_df.duration <= 60)).mean() * 100

97.42784903545073

In [8]:
# filtering the dataset to remove outliers
mar_df = mar_df[(mar_df.duration >= 1) & (mar_df.duration <= 60)]

mar_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333


In [12]:
# selecting the features
mar_cat = ["PULocationID", "DOLocationID"]

#convertng the categorical features to str datatype
mar_df[mar_cat] = mar_df[mar_cat].astype(str)

# intializing DictVectorzer
dv = DictVectorizer()

# converting the catorical train data to dictionary data type
mar_train_dict = mar_df[mar_cat].to_dict(orient = "records")
mar_X_train = dv.fit_transform(mar_train_dict)

In [13]:
# target variable for january data
mar_target = "duration"
mar_y_train = mar_df[mar_target].values

In [14]:
# intializing linear regression moodel
lr = LinearRegression()
# fitting the train data into the linear regression model
lr.fit(mar_X_train, mar_y_train)

# predicting the validation target
mar_y_pred = lr.predict(mar_X_train)

# estimating the error between the val original target variable with the val predicted  target variable using rmse.
mean_squared_error(mar_y_train, mar_y_pred, squared = False)

8.158679489031586

In [17]:
with open("lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
pred = {
    "prediction" : mar_y_pred
}
df = pd.DataFrame(pred)
df.head()

,prediction
0,16.672064
1,28.668098
2,12.533435
3,12.686599
4,10.446526


In [21]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df_result = df[["ride_id", "prediction"]]
df_result

,ride_id,prediction
0,2023/03_0,16.672064
1,2023/03_1,28.668098
2,2023/03_2,12.533435
3,2023/03_3,12.686599
4,2023/03_4,10.446526
...,...,...
3316211,2023/03_3316211,12.395406
3316212,2023/03_3316212,19.154061
3316213,2023/03_3316213,11.640608
3316214,2023/03_3316214,13.182929


In [24]:
output_file = "output.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [26]:
import os
# Get the size of the saved file
file_size = os.path.getsize(output_file)
file_size_mb = file_size / (1024 * 1024)  # Convert bytes to megabytes
print(f'Size of the output file: {file_size_mb:.2f} MB')

Size of the output file: 65.44 MB
